In [1]:
# od pair
from env import env
import pandas as pd

# here requests
from haversine import haversine, Unit
import flexpolyline as fp
import urllib.request
import json, sys

# co2 emission - top down
import instances.emission as gcc

# map plot
from bs4 import BeautifulSoup as bs
import gmplot

In [2]:
filtered = pd.read_csv('datasets/new_bike/filtered_dataset_gas.csv')
odDf = pd.read_csv('datasets/new_bike/set_id_coord.csv')

### Get od pair

In [3]:
# coletar par od qualquer para teste

ind = 0 # filtered route index
#od = odDf.loc[odDf['setID'] == filtered.iloc[ind]['setID']]

od = odDf.loc[odDf['setID'] == 302]

origin = od['departure_coord'].to_string(index=False).split(":")
origin = f"{origin[1]},{origin[0]}"

destination = od['arrival_coord'].to_string(index=False).split(":")
destination = f"{destination[1]},{destination[0]}"

float_origin = list(map(lambda x: float(x),origin.split(',')))
float_destination = list(map(lambda x: float(x),destination.split(',')))

print(origin)
print(destination)
print(float_origin)
print(float_destination)

#-0.045392:51.407064,0.061901:51.470691

51.407064,-0.045392
51.470691,0.061901
[51.407064, -0.045392]
[51.470691, 0.061901]


### Here requests

Car co2 and fuel consumption link: https://developer.here.com/documentation/routing-api/dev_guide/topics/use-cases/calculate-fuel-consumption.html

In [5]:
def route_midpoint(route_length,zipList):#,latList,lngList):
    middle = delta = route_length/2
    before = after = None
    dist = 0

    for i in range(len(zipList)-1):
        dist += haversine(zipList[i],zipList[i+1], unit=Unit.METERS)
        sub = abs(middle-dist)
        if sub < delta:
            delta = sub
            before = zipList[i+1]
        if sub > delta:
            after = zipList[i+1]
            break

    midpointCoord = ((before[0]+after[0])/2,(before[1]+after[1])/2)
    midpoint = f'{midpointCoord[0]},{midpointCoord[1]}'

    return midpoint,midpointCoord

# get closest bike share point on coordinate
def closest(coord: list): #### need refactor!!
    data = pd.read_csv('results/bike-results/result.csv')
    dist = sys.float_info.max
    waypoint = ()
    for ind,row in data.iterrows():
        distTest = haversine(coord,(row['station_lat'],row['station_lng']))
        if distTest < dist:
            dist = distTest
            waypoint = (row['station_lat'],row['station_lng'])
    return waypoint

def closest_list(coord: list, data: list):
    dist = sys.float_info.max
    waypoint = ()
    for i in data:
        distTest = haversine(coord,(float(i[0]),float(i[1])))
        if distTest < dist:
            dist = distTest
            waypoint = (float(i[0]),float(i[1]))
    return waypoint

In [7]:
# for pedestrian: https://developer.here.com/documentation/routing-api/dev_guide/topics/pedestrian-defined.html
#hereKey = env('HERE_API_KEY')

def car_routing(hereKey,strCoord,maxAlternatives=0):
    query = f'destination={strCoord[1]}&origin={strCoord[0]}'
    
    fuelConsumption = '&fuel[type]=diesel&fuel[freeflowspeedtable]=0,0,60,0.1,80,0.12&fuel[trafficspeedtable]=0,0,60,0.1,80,0.12&fuel[additionalconsumption]=1.8&fuel[ascent]=0.1'

    url = 'https://router.hereapi.com/v8/routes?transportMode=%s%s&%s&alternatives=%d&return=summary,polyline&apikey=%s' % ('car',fuelConsumption,query,maxAlternatives,hereKey)
    response = urllib.request.urlopen(url).read().decode('utf-8')
    return [json.loads(response)]

def transit_routing(hereKey,strCoord,maxAlternatives=0):
    query = f'destination={strCoord[1]}&origin={strCoord[0]}'
    
    url = 'https://intermodal.router.hereapi.com/v8/routes?%s&transit[enable]=entireRoute&rented[enable]=&taxi[enable]=&vehicle[enable]=&alternatives=%d&return=travelSummary,polyline&apiKey=%s' % (query,maxAlternatives,hereKey)
    response = urllib.request.urlopen(url).read().decode('utf-8')
    return [json.loads(response)]

def bicycle_routing(hereKey,strCoord,floatCoord,maxAlternatives=0):

    newOrigin = closest(floatCoord[0])
    newDestination = closest(floatCoord[1])

    mode = ['pedestrian','bicycle','pedestrian']

    query = [
        f'destination={newOrigin[0]},{newOrigin[1]}&origin={strCoord[0]}',
        f'destination={newDestination[0]},{newDestination[1]}&origin={newOrigin[0]},{newOrigin[1]}',
        f'destination={strCoord[1]}&origin={newDestination[0]},{newDestination[1]}'
    ]

    responses = []

    for i in range(len(mode)):
        if mode[i] == 'pedestrian': altNum = 0
        else: altNum = maxAlternatives
        url = 'https://router.hereapi.com/v8/routes?transportMode=%s&%s&alternatives=%d&return=summary,polyline&apikey=%s' % (mode[i],query[i],altNum,hereKey)
        response = urllib.request.urlopen(url).read().decode('utf-8')
        responses.append(json.loads(response))

    return responses,newDestination

def simple_bicycle_routing(hereKey,strCoord,maxAlternatives=0):
    query = f'destination={strCoord[1]}&origin={strCoord[0]}'

    url = 'https://router.hereapi.com/v8/routes?transportMode=%s&%s&alternatives=%d&return=summary,polyline&apikey=%s' % ('bicycle',query,maxAlternatives,hereKey)
    response = urllib.request.urlopen(url).read().decode('utf-8')
    return [json.loads(response)]

def response_reader(response,midpoint=False): # update to return dict
    polyline = []
    distance = 0
    duration = 0

    for section in response['routes'][0]['sections']:
        try:
            distance += section['summary']['length']
            duration += section['summary']['duration']
        except:
            distance += section['travelSummary']['length']
            duration += section['travelSummary']['duration']
        polyline += fp.decode(section['polyline'])

    if midpoint: return duration,distance,polyline,route_midpoint(distance,polyline)
    return duration,distance,polyline


In [1]:
def car_price_estimate(dist,dura):
    min_cost,km_cost,reserva,tarifa_base = 0.25,1.3,1.2,5
    return tarifa_base + (min_cost * (dura/60)) + (km_cost * (dist / 1000)) + reserva

def bicycle_price_estimate(dura):
    tarifa_base,minuto_custo = 3,0.1
    return tarifa_base + (minuto_custo * (dura / 60))

In [ ]:
print(bicycle_price_estimate)

In [9]:
def polylineToCoord(tracer):
    tracer = tracer.replace(':',',').split(';')
    coordinates = []
    for i in tracer:
        coord = i.split(',')
        coordinates.append(((float(coord[0])),(float(coord[1]))))
    return coordinates

def map_plot(origin: str,og_polyline: list = None,new_polyline_list: list = [],color_list: list = [],fileName: str = '') -> None:
    origin_coord = origin.split(',')
    outputMap = gmplot.GoogleMapPlotter(float(origin_coord[0]),float(origin_coord[1]), 12)
    mapUrl = f'results/map_plot/{fileName}.html'

    colors = ['black','grey','pink','orange','cyan','violet']

    if og_polyline != None:
        #for polyline in og_polyline:
        for i in range(len(og_polyline)):
            lats = [t[1] for t in og_polyline[i]]
            lngs = [t[0] for t in og_polyline[i]]
            
            outputMap.plot(lats,lngs, color=colors[i], edge_width=3)

    for i in range(len(new_polyline_list)):

        new_lats = [t[0] for t in new_polyline_list[i]]
        new_lngs = [t[1] for t in new_polyline_list[i]]

        outputMap.plot(new_lats, new_lngs, color=color_list[i], edge_width=3)

    outputMap.draw(mapUrl)

    page = open(mapUrl)
    soup = bs(page.read(),"html.parser")
    srcKey = f"https://maps.googleapis.com/maps/api/js?key={env('GOOGLE_MAPS_KEY')}&libraries=visualization&sensor=true_or_false"
    line = soup.find('script',type="text/javascript")
    line['src'] = srcKey

    with open(mapUrl, "wb") as f_output:
        f_output.write(soup.prettify("utf-8")) 

In [11]:
def main(coordDF):
    dataframe = pd.DataFrame(columns=["setID","duration","distance","emission","price","modal"])
    transit_price = 4
    api_key = 1
    here_api = env(f"HERE_API_KEY_{api_key}")
    for index, row in coordDF.iterrows():
        if index % 3 == 0 and index != 0:
            if api_key == 3: api_key = 1
            else: api_key += 1 
            here_api = env(f"HERE_API_KEY_{api_key}")
            print("key used:",f"HERE_API_KEY_{api_key}")
        try:

            # coleta do par od
            origin = row['departure_coord'].split(":")
            origin = f"{origin[1]},{origin[0]}"

            destination = row['arrival_coord'].split(":")
            destination = f"{destination[1]},{destination[0]}"

            coords = (origin,destination)

            # full modal
            response_full_car = car_routing(here_api,coords)
            response_full_transit = transit_routing(here_api,coords)
            response_full_bicycle = simple_bicycle_routing(here_api,coords)

            # full modal results
            duration_full_car,distance_full_car,polyline_full_car,car_midpoint = response_reader(*response_full_car,True)
            duration_full_transit,distance_full_transit,polyline_full_transit,transit_midpoint = response_reader(*response_full_transit,True)
            duration_full_bicycle,distance_full_bicycle,polyline_full_bicycle,bicycle_midpoint = response_reader(*response_full_bicycle,True)

            car_price = car_price_estimate(distance_full_car,duration_full_car)
            bike_price = bicycle_price_estimate(duration_full_bicycle)
            
            m3 = ((distance_full_transit / 1000) / 2.5) / 1000
            transit_emission = gcc.gcc_emission(m3,"diesel","solid_liquid","diesel")
            car_emission = response_full_car[0]['routes'][0]['sections'][0]['summary']['co2Emission']

            new_row_0 = pd.DataFrame({"setID":[row['setID']],"duration":[duration_full_car],"distance":[distance_full_car],"emission":[car_emission],"price":[car_price],"modal":["car"]})
            new_row_1 = pd.DataFrame({"setID":[row['setID']],"duration":[duration_full_transit],"distance":[distance_full_transit],"emission":[transit_emission],"price":[transit_price],"modal":["transit"]})
            new_row_2 = pd.DataFrame({"setID":[row['setID']],"duration":[duration_full_bicycle],"distance":[distance_full_bicycle],"emission":[0],"price":[bike_price],"modal":["bicycle"]})

            dataframe = pd.concat([dataframe,new_row_0],ignore_index=True)
            dataframe = pd.concat([dataframe,new_row_1],ignore_index=True)
            dataframe = pd.concat([dataframe,new_row_2],ignore_index=True)

            # car main multimodal
            response_car = car_routing(here_api,(origin,car_midpoint[0]))
            response_car_transit = transit_routing(here_api,(car_midpoint[0],destination))
            response_car_bicycle = simple_bicycle_routing(here_api,(car_midpoint[0],destination))

            # transit main multimodal
            response_transit = transit_routing(here_api,(origin,transit_midpoint[0]))
            response_transit_car = car_routing(here_api,(transit_midpoint[0],destination))
            response_transit_bicycle = simple_bicycle_routing(here_api,(transit_midpoint[0],destination))

            # bicycle main multimodal
            response_bicycle = simple_bicycle_routing(here_api,(origin,bicycle_midpoint[0]))
            response_bicycle_car = car_routing(here_api,(bicycle_midpoint[0],destination))
            response_bicycle_transit = transit_routing(here_api,(bicycle_midpoint[0],destination))

            # car main multimodal results
            duration_car,distance_car,polyline_car = response_reader(*response_car)
            duration_car_transit,distance_car_transit,polyline_car_transit = response_reader(*response_car_transit)
            duration_car_bicycle,distance_car_bicycle,polyline_car_bicycle = response_reader(*response_car_bicycle)

            car_price = car_price_estimate(distance_car,duration_car)
            bike_price = bicycle_price_estimate(duration_car_bicycle)

            m3 = ((distance_car_transit / 1000) / 2.5) / 1000
            transit_emission = gcc.gcc_emission(m3,"diesel","solid_liquid","diesel")
            car_emission = response_full_car[0]['routes'][0]['sections'][0]['summary']['co2Emission']

            new_row_3 = pd.DataFrame({"setID":[row['setID']],"duration":[duration_car+duration_car_transit],"distance":[distance_car+distance_car_transit],"emission":[car_emission+transit_emission],"price":[car_price+transit_price],"modal":["car-transit"]})
            new_row_4 = pd.DataFrame({"setID":[row['setID']],"duration":[duration_car+duration_car_bicycle],"distance":[distance_car+distance_car_bicycle],"emission":[car_emission],"price":[car_price+bike_price],"modal":["car-bicycle"]})

            dataframe = pd.concat([dataframe,new_row_3],ignore_index=True)
            dataframe = pd.concat([dataframe,new_row_4],ignore_index=True)

            # transit main multimodal results
            duration_transit,distance_transit,polyline_transit = response_reader(*response_transit)
            duration_transit_car,distance_transit_car,polyline_transit_car = response_reader(*response_transit_car)
            duration_transit_bicycle,distance_transit_bicycle,polyline_transit_bicycle = response_reader(*response_transit_bicycle)

            car_price = car_price_estimate(distance_transit_car,duration_transit_car)
            bike_price = bicycle_price_estimate(duration_transit_bicycle)

            m3 = ((distance_transit / 1000) / 2.5) / 1000
            transit_emission = gcc.gcc_emission(m3,"diesel","solid_liquid","diesel")
            car_emission = response_full_car[0]['routes'][0]['sections'][0]['summary']['co2Emission']

            new_row_5 = pd.DataFrame({"setID":[row['setID']],"duration":[duration_transit+duration_transit_car],"distance":[distance_transit+distance_transit_car],"emission":[transit_emission+car_emission],"price":[transit_price+car_price],"modal":["transit-car"]})
            new_row_6 = pd.DataFrame({"setID":[row['setID']],"duration":[duration_transit+duration_transit_bicycle],"distance":[distance_transit+distance_transit_bicycle],"emission":[transit_emission],"price":[transit_price+bike_price],"modal":["transit-bicycle"]})

            dataframe = pd.concat([dataframe,new_row_5],ignore_index=True)
            dataframe = pd.concat([dataframe,new_row_6],ignore_index=True)

            # bicycle main multimodal results
            duration_bicycle,distance_bicycle,polyline_bicycle = response_reader(*response_bicycle)
            duration_bicycle_car,distance_bicycle_car,polyline_bicycle_car = response_reader(*response_bicycle_car)
            duration_bicycle_transit,distance_bicycle_transit,polyline_bicycle_transit = response_reader(*response_bicycle_transit)

            car_price = car_price_estimate(distance_bicycle_car,duration_bicycle_car)
            bike_price = bicycle_price_estimate(duration_bicycle)

            m3 = ((distance_bicycle_transit / 1000) / 2.5) / 1000
            transit_emission = gcc.gcc_emission(m3,"diesel","solid_liquid","diesel")
            car_emission = response_full_car[0]['routes'][0]['sections'][0]['summary']['co2Emission']

            new_row_7 = pd.DataFrame({"setID":[row['setID']],"duration":[duration_bicycle+duration_bicycle_car],"distance":[distance_bicycle+distance_bicycle_car],"emission":[car_emission],"price":[bike_price+car_price],"modal":["bicycle-car"]})
            new_row_8 = pd.DataFrame({"setID":[row['setID']],"duration":[duration_bicycle+duration_bicycle_transit],"distance":[distance_bicycle+distance_bicycle_transit],"emission":[transit_emission],"price":[bike_price+transit_price],"modal":["bicycle-transit"]})
            
            dataframe = pd.concat([dataframe,new_row_7],ignore_index=True)
            dataframe = pd.concat([dataframe,new_row_8],ignore_index=True)

            print(index,coords,"finalizado")
        
        except Exception as e:
            print(f"erro no index {index}")
            print(e)
            new_row_error = pd.DataFrame({"setID":[row['setID']],"duration":[None],"distance":[None],"modal":['error']})
            dataframe = pd.concat([dataframe,new_row_error])
    dataframe.to_csv("results/responses.csv")

In [12]:
main(odDf)

C:\Users\Camilo\AppData\Local\Temp\ipykernel_16808\249431167.py:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframe = pd.concat([dataframe,new_row_0],ignore_index=True)


0 ('51.533778,-0.2677', '51.57419,-0.46799') finalizado
1 ('51.58165,0.04329', '51.566671,0.004042') finalizado
2 ('51.49815,-0.407351', '51.459689,-0.618729') finalizado
key used: HERE_API_KEY_2
3 ('51.53691,-0.00175', '51.558251,0.06236') finalizado
4 ('51.43737,0.01641', '51.448036,0.100414') finalizado
5 ('51.486866,0.05292', '51.49816,0.155019') finalizado
key used: HERE_API_KEY_3
6 ('51.43961,-0.13137', '51.368343,-0.140186') finalizado
erro no index 7
list index out of range
8 ('51.43099,-0.03831', '51.458881,-0.195708') finalizado
key used: HERE_API_KEY_1
9 ('51.437391,-0.229127', '51.346211,-0.375245') finalizado
10 ('51.614032,-0.041873', '51.5827,-0.19678') finalizado
11 ('51.44875,-0.222431', '51.36979,-0.31404') finalizado
key used: HERE_API_KEY_2
12 ('51.550028,-0.371718', '51.49416,-0.48164') finalizado
13 ('51.570791,0.031789', '51.495947,0.002322') finalizado
14 ('51.432679,0.072682', '51.400427,0.11317') finalizado
key used: HERE_API_KEY_3
15 ('51.52531,0.096989', '51

In [10]:

coords = (origin,destination)
float_coords = (float_origin,float_destination)
here_key = env('HERE_API_KEY_1')

response_full_car = car_routing(here_key,coords) # full car
response_full_transit = transit_routing(here_key,coords) # full transit
response_full_bicycle = simple_bicycle_routing(here_key,coords) # full bicycle

duration_full_car,distance_full_car,polyline_full_car,car_midpoint = response_reader(*response_full_car,True)
duration_full_transit,distance_full_transit,polyline_full_transit,transit_midpoint = response_reader(*response_full_transit,True)
duration_full_bicycle,distance_full_bicycle,polyline_full_bicycle,bicycle_midpoint = response_reader(*response_full_bicycle,True)

response_car = car_routing(here_key,(origin,car_midpoint[0]))
response_car_transit = transit_routing(here_key,(car_midpoint[0],destination))
response_car_bicycle = simple_bicycle_routing(here_key,(car_midpoint[0],destination))

response_transit = transit_routing(here_key,(origin,transit_midpoint[0]))
response_transit_car = car_routing(here_key,(transit_midpoint[0],destination))
response_transit_bicycle = simple_bicycle_routing(here_key,(transit_midpoint[0],destination))

response_bicycle = simple_bicycle_routing(here_key,(origin,bicycle_midpoint[0]))
response_bicycle_car = car_routing(here_key,(bicycle_midpoint[0],destination))
response_bicycle_transit = transit_routing(here_key,(bicycle_midpoint[0],destination))

duration_car,distance_car,polyline_car = response_reader(*response_car)
duration_car_transit,distance_car_transit,polyline_car_transit = response_reader(*response_car_transit)
duration_car_bicycle,distance_car_bicycle,polyline_car_bicycle = response_reader(*response_car_bicycle)

duration_transit,distance_transit,polyline_transit = response_reader(*response_transit)
duration_transit_car,distance_transit_car,polyline_transit_car = response_reader(*response_transit_car)
duration_transit_bicycle,distance_transit_bicycle,polyline_transit_bicycle = response_reader(*response_transit_bicycle)

duration_bicycle,distance_bicycle,polyline_bicycle = response_reader(*response_bicycle)
duration_bicycle_car,distance_bicycle_car,polyline_bicycle_car = response_reader(*response_bicycle_car)
duration_bicycle_transit,distance_bicycle_transit,polyline_bicycle_transit = response_reader(*response_bicycle_transit)


In [19]:
id = float(od['setID'].values)
doc = filtered[filtered['setID'] == id]

treated_polylines = []

for row in doc.iterrows():
    treated_polylines.append(polylineToCoord(row[1]['coordinates']))

In [20]:
#car: blue
#transit: red
#bicycle: green

## walk: white

# full modal
map_plot(origin,treated_polylines,[polyline_full_car],["blue"],"full_car")
map_plot(origin,treated_polylines,[polyline_full_transit],["red"],"full_transit")
map_plot(origin,treated_polylines,[polyline_full_bicycle],["green"],"full_bicycle")

# car main
map_plot(origin,treated_polylines,[polyline_car,polyline_car_transit],['blue','red'],'car_transit')
map_plot(origin,treated_polylines,[polyline_car,polyline_car_bicycle],['blue','green'],'car_bicycle')

# transit main
map_plot(origin,treated_polylines,[polyline_transit,polyline_transit_car],['red','blue'],'transit_car')
map_plot(origin,treated_polylines,[polyline_transit,polyline_transit_bicycle],['red','green'],'transit_bicycle')

# bicycle main
map_plot(origin,treated_polylines,[polyline_bicycle,polyline_bicycle_car],['green','blue'],'bicycle_car')
map_plot(origin,treated_polylines,[polyline_bicycle,polyline_bicycle_transit],['green','red'],'bicycle_transit')


## Test

In [29]:
# car price test

test_origin = 'origin-coord'
test_destination = 'destination-coord'

response_test = car_routing((test_origin,test_destination))
print(response_test)

duration_test,distance_test,polyline_test = response_reader(*response_test)

map_plot(test_origin,None,[polyline_test],['blue'],'car_test_price')

print(duration_test,distance_test/60)
print(car_price_estimate(distance_test,duration_test))

{'routes': [{'id': '2da54301-3bce-4094-92a5-73a70f0af16a', 'sections': [{'id': '4bd6edef-e13a-4cdc-8313-3f279764985b', 'type': 'vehicle', 'departure': {'time': '2023-10-26T11:01:59-03:00', 'place': {'type': 'place', 'location': {'lat': -1.4727449, 'lng': -48.4515593, 'elv': -19.0}, 'originalLocation': {'lat': -1.4727494, 'lng': -48.4515741}}}, 'arrival': {'time': '2023-10-26T11:19:06-03:00', 'place': {'type': 'place', 'location': {'lat': -1.4350656, 'lng': -48.4776198}, 'originalLocation': {'lat': -1.4350718, 'lng': -48.4776241}}}, 'summary': {'duration': 1027, 'length': 7624, 'consumption': 2.33, 'baseDuration': 833, 'co2Emission': 6.284}, 'polyline': 'BGxu85Ct-nt8CmCVkXvMwC4D4DsEgFgF0F0FkN0KsY0UgPkNwC8BkD8BoGkDoVwR0FsEgFkDsJ0FopBwWkcsO8VwM4NkIsxBoa4hB4S4DwC0ZkN0K0F8LsJkXwMgZ8LoVoLoVwMwbsO4XwM8V8Loa4NwWwMgZwMgK0F8asO4SgK0e8QwWwMoLoG4XwM0K0FgUwHgPgFwR8B8GU8QUgKT8QjD8LjD8GvCwH7BkIvCwHvCkIjDwH3DkIrEwHrE0K7Gwb3S8anQ4IrEsOnG8VzK8LrEwMrE0e_EoL7BoGnB8kB_EwRvCsY3DopBjD0Z7B4NToLT0KToLU8BoB8BU8

In [88]:
# transit route dataset value test

data = filtered['midpoint'].to_list()
data = list(map(lambda x: x.split(","),data))
hope = closest_list(transit_midpoint[1],data)

dataf_hope = filtered[filtered['midpoint'] == f'{hope[0]},{hope[1]}']['coordinates'].values
treated_hope = polylineToCoord(*dataf_hope)

map_plot(origin,treated_hope,[polyline_full_transit],["red"],"transit_value_test")

[(-0.2677, 51.533778), (-0.2677, 51.53326), (-0.26763, 51.53323), (-0.26758, 51.53319), (-0.26758, 51.53314), (-0.26656, 51.53315), (-0.26555, 51.53321), (-0.26507, 51.5332), (-0.26479, 51.53319), (-0.26477, 51.53324), (-0.26461, 51.53337), (-0.26426, 51.53347), (-0.26347, 51.53367), (-0.26295, 51.53381), (-0.26234, 51.534), (-0.26189, 51.53411), (-0.2617, 51.53415), (-0.26148, 51.53418), (-0.26128, 51.53418), (-0.26111, 51.53416), (-0.26094, 51.53412), (-0.26076, 51.53407), (-0.2607, 51.53401), (-0.26064, 51.53393), (-0.26059, 51.53385), (-0.26054, 51.53375), (-0.26033, 51.53321), (-0.26025, 51.53302), (-0.2602, 51.53284), (-0.26018, 51.53268), (-0.26015, 51.532), (-0.26015, 51.53198), (-0.2601, 51.5316), (-0.26005, 51.53123), (-0.25997, 51.53062), (-0.25997, 51.53053), (-0.25988, 51.53013), (-0.25977, 51.52986), (-0.2596, 51.52946), (-0.25939, 51.52899), (-0.25933, 51.52891), (-0.25928, 51.52892), (-0.25923, 51.52891), (-0.25919, 51.52889), (-0.25918, 51.52886), (-0.25919, 51.52883),